In [1]:
tsobc_file = "/home/benr/wqmodels/ssm/hyd-test/model_input/ssm_tsobc.dat"
out_file = "data/fvcom_tsobc_2014.nc"

import numpy as np
from netCDF4 import Dataset

In [2]:
time_lines = []
temp_data = []
salt_data = []
with open(tsobc_file) as f:
    t_line = next(f)
    while True:
        time_lines.append(t_line)
        n0, nlay, *layers = np.loadtxt([next(f)], dtype=int)
        temp_data.append(np.loadtxt([next(f) for l in range(n0)]))
        salt_data.append(np.loadtxt([next(f) for l in range(n0)]))
        # Look for the next time index, which might fail if we're at the end-of-file
        try:
            t_line = next(f)
        except StopIteration:
            break

times = np.loadtxt(time_lines, comments='!')
temp_data = np.array(temp_data)
# Extract and remove node numbers
nodes = temp_data[0,:,0]
temp_data = temp_data[:,:,1:]
salt_data = np.array(salt_data)[:,:,1:]

temp_data.shape

(14, 87, 10)

In [3]:
cdf = Dataset(out_file, "w")
# Extract the first comment line from time_lines and use as the file title
comment = time_lines[0].strip().split('!')
if len(comment) > 1:
    cdf.comment = comment[1]
time_dim = cdf.createDimension("time", len(times))
node_dim = cdf.createDimension("node", n0)
siglay_dim = cdf.createDimension("siglay", nlay)

time_var = cdf.createVariable("time", "f4", ("time",))
time_var.unit = "days"
node_var = cdf.createVariable("node", "i4", ("node",))
temp_var = cdf.createVariable("temp", "f4", ("time","node","siglay"))
salt_var = cdf.createVariable("salt", "f4", ("time","node","siglay"))

cdf['time'][:] = times
cdf['node'][:] = nodes
cdf['temp'][:] = temp_data
cdf['salt'][:] = salt_data

cdf

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    comment: 200 m depth obc
    dimensions(sizes): time(14), node(87), siglay(10)
    variables(dimensions): float32 time(time), int32 node(node), float32 temp(time, node, siglay), float32 salt(time, node, siglay)
    groups: 

In [4]:
cdf.close()